In [1]:
import cobra
import libsbml
from cobra.core import Metabolite, Reaction
import pandas as pd
from cobra import flux_analysis
from Functions import checkProtonFluxes
ModelF = cobra.io.read_sbml_model("CAM_12P_ME_Model.xml")
ModelF.solver="glpk"
cobra.flux_analysis.pfba(ModelF)

,fluxes,reduced_costs
PRO_PROTON_vc1,0.000000,2.0
Ca_tx1,0.000000,2.0
H2O_xc1,0.000000,-2.0
sCIT_biomass1,0.000000,2.0
ACETYLGLUTKIN_RXN_p1,0.000106,-2.0
...,...,...
PYR_mc8_channel,0.000000,2.0
PYR_mc9_channel,0.000000,0.0
PYR_mc10_channel,0.000000,0.0
PYR_mc11_channel,0.000000,6.0


In [2]:
ModelF3 = ModelF.copy()

In [3]:
#Reintroduce NADP-ME:NAD-ME and cPPDK:pPPDK constraints when needed
#Constrain a flux ratio between NAD-ME and NADP-ME of 8:1 according to enzyme activity measurements in Kalanchoë and Dever et al. 2015
for i in range(1,13):
    new_constraint1 = ModelF3.problem.Constraint(
        8*(ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i)).flux_expression + ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i)).flux_expression) - ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint1)
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)

In [4]:
#Constrain a flux ratio between cPPDK and pPPDK of 2:1 according to Kondo et al. 2000 and Dever et al. 2015
for i in range(1,13):
    new_constraint2 = ModelF3.problem.Constraint(
        2*ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint2)

sol3 = flux_analysis.parsimonious.pfba(ModelF3)

In [5]:
#Check current NAD-ME:NADP-ME and cPPDK:pPPDK ratio
NADP_ME=0
for i in range(1,13):
    rxn=ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i))
    NADP_ME=NADP_ME+rxn.flux
    rxn=ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i))
    NADP_ME=NADP_ME+rxn.flux

NAD_ME=0
for i in range(1,13):
    rxn=ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i))
    NAD_ME=NAD_ME+rxn.flux
ratioME=NAD_ME/NADP_ME
print("NAD-ME:NADP-ME= "+str(ratioME))

cPPDK=0
for i in range(1,13):
    rxn=ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i))
    cPPDK=cPPDK+rxn.flux
    
pPPDK=0
for i in range(1,13):
    rxn=ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i))
    pPPDK=pPPDK+rxn.flux
ratioPPDK=cPPDK/pPPDK
print("---------------")
print("cPPDK:pPPDK= "+str(ratioPPDK))

NAD-ME:NADP-ME= 8.0
---------------
cPPDK:pPPDK= 2.0


In [6]:
#Sensitivity analysis - Rubisco carboxylase/oxygenase ratio

#Constrain different Rubisco carboxylase and oxygenase ratios (Vc/Vo) 
#Examined ratios: unconstrained (see notebook ME_refinement_and_simulations), 2, 3, 5, 10, 1000
VcVoratio = 2 #change 2 to the ratio that needs to be checked
for i in range(1,7):
    new_constraint3 = ModelF3.problem.Constraint(
        VcVoratio*ModelF3.reactions.get_by_id("RXN_961_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint3)
    
#Set H_mc to carry 0 flux as this was included as a hypothetical reaction
for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0

sol3 = flux_analysis.parsimonious.pfba(ModelF3)
checkProtonFluxes(ModelF3,tag="VcVoRatio")

In [7]:
#Sensitivity analysis - Maintencance cost
#Remove previous constraints on maintenance cost 
for i in range(1,13):
    ModelF3.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = 1000
    
    ModelF3.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = 1000
    
    ModelF3.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = 1000
    
    ModelF3.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = 1000

PPFD = 100                        #light intensity of the model
ATPase = (0.0049*PPFD) + 2.7851   #non-growth assocaited maintenance (NGAM) cost based on light - see Topfer et al 2020 Supplemental information section 1.2.3

#Uncomment to check flux distribution
#ATPase = ((0.0049*PPFD) + 2.7851)*1.5 #Increase maintenance cost by 50%
#ATPase = ((0.0049*PPFD) + 2.7851)*0.5 #Decrease maintenance cost by 50%
for i in range(1,13):
    ModelF3.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = ATPase
    ModelF3.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = ATPase
    
    #Setting NADPH demand to 1/3 of ATP demand and distributing this demand to cytosol, plastid and mitochondria based on Cheung et al 2013 (doi: 10.1111/tpj.12252)
    ModelF3.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = ATPase/9
    ModelF3.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF3.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = ATPase/9
    ModelF3.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF3.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = ATPase/9
    ModelF3.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = ATPase/9

sol3 = flux_analysis.parsimonious.pfba(ModelF3)
checkProtonFluxes(ModelF3,tag="NGAM")

In [8]:
#Sensitivity analysis - NAD-ME:NADP-ME ratios

MEratio=8 #Adjust value of MEratio (8) to examine sensitivity of NADP-ME:NAD-ME constraints on key proton reactions
          #Comment to leave unconstrained

for i in range(1,13):
    new_constraint1 = ModelF3.problem.Constraint(
        MEratio*(ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i)).flux_expression + ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i)).flux_expression) - ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint1)

#Set H_mc to carry 0 flux as this was included as a hypothetical reaction
for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0
  
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
checkProtonFluxes(ModelF3,tag="ME_ratio")

In [9]:
#Sensitivity analysis - cPPDK:pPPDK ratios

PPDKratio=2 #Adjust value of PPDKratio (2) to examine sensitivity of cPPDK:pPPDK constraints on key proton reactions
            #Comment to leave unconstrained

for i in range(1,13):
    new_constraint2 = ModelF3.problem.Constraint(
        PPDKratio*ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint2)
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
checkProtonFluxes(ModelF3,tag="PPDK_ratio")

In [10]:
#Sensitivity analysis - Decarboxylating enzyme

for i in range(1,13):    
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).upper_bound = 1000
    
#Set H_mc to carry 0 flux as this was included as a hypothetical reaction
for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
checkProtonFluxes(ModelF3,tag="PEPCK")

In [ ]:
from Functions import customFVA
fva_sol=customFVA(ModelF3,rxnlist = ["MAL_PROTON_rev_vc1","MAL_PROTON_rev_vc2", "MAL_PROTON_rev_vc3", "MAL_PROTON_rev_vc4", "MAL_PROTON_rev_vc5", "MAL_PROTON_rev_vc6", "Pi_PROTON_mc1","Pi_PROTON_mc2","Pi_PROTON_mc3","Pi_PROTON_mc4","Pi_PROTON_mc5", "Pi_PROTON_mc6", "GAPOXNPHOSPHN_RXN_c1", "GAPOXNPHOSPHN_RXN_c2","GAPOXNPHOSPHN_RXN_c3","GAPOXNPHOSPHN_RXN_c4","GAPOXNPHOSPHN_RXN_c5","GAPOXNPHOSPHN_RXN_c6"])

fva_sol=pd.DataFrame(fva_sol)
output_file="FVA.xlsx"
fva_sol.to_excel(output_file, index=False)